<a href="https://colab.research.google.com/github/valmirf/mineracao_textual/blob/main/NER/06a_ExtracaoInformacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Named entity recognition (NER) <a id='ner'></a>

Named entity recognition (NER) é uma das principais tarefas em projetos de recuperação e extração de informação em textos. Muitas tarefas se iniciam a partir da detecção de entidades nomeadas, como a extração de relações por exemplo. 

Em NER, as diferentes entidades nomeadas extraídas são agrupadas por tipo. Por exemplo, "pessoa", "organização", "local", "país" etc. No spaCy, existem muitos [tipos diferentes](https://spacy.io/api/annotation#named-entities) de entidades nomeadas que ele pode extrair com modelos pré-treinados.

As entidades nomeadas no spaCy estão disponíveis como propriedade `ents` de um` Doc`. O `.label_` nos diz o tipo de entidade nomeada.

In [ ]:
#instalando bibliotecas
!pip install spacy
!spacy download pt

In [ ]:
import os
import spacy
import numpy as np
import pandas as pd

In [ ]:
nlp = spacy.load('pt')
doc = nlp("Eu estarei em Recife próxima semana. Será que levo roupa para frio?")
for ent in doc.ents:
    print(ent.text,ent.label_)

In [ ]:
doc = nlp("Geraldo Júlio é o prefeito de Recife.")
for ent in doc.ents:
    print(ent.text,ent.label_)

In [ ]:
#explain também funciona para entidades
spacy.explain('LOC')

### Exercício 1

Extraia todas as entidades nomeadas da variável `review`.

In [ ]:
# Texto
review = '''Este é sem dúvida o pior filme que eu já vi. E acredite em mim, eu vi muitos filmes. A reviravolta inacreditável que o filme faz - passando de um extremamente mau filme "Formas de vida alienígenas habitam a terra", com um filme que tenta espalhar um arquicristiano "O dia do julgamento está próximo, buscar Jesus ou queimar por toda a eternidade em as dívidas ardentes do inferno "mensagem - deixou-me atordoado depois de ter sido atormentado por 85 minutos. Até mesmo os cristãos religiosos devem se envergonhar ou ficar furiosos ao ver suas crenças postadas dessa maneira. Eu não sabia o que fazer comigo quando assisti a atuação horrível que poderia ter sido realizada por crianças de 7 anos de idade. Simplesmente repugnante. Eu não sou cristão nem muito religioso. Mas se eu estivesse, não teria mais medo do Inferno. Rich Christiano mostrou ser algo muito pior.'''

doc = nlp(review)
for ent in doc.ents:
    print(ent.text,' - ',ent.label_,' - ',spacy.explain(ent.label_))

### Exercício 2
Utilizar a lib do python `wikipedia`, para baixar o conteúdo da página referente ao cantor Luiz Gonzaga no wikipedia em português, e analise quais as entidades presentes nas 10 primeiras sentenças do texto.

In [ ]:
!pip install wikipedia

In [ ]:
import wikipedia
wikipedia.set_lang("pt")
p = wikipedia.page("Luiz Gonzaga")

print(p.url)
print(p.title)

content = p.content 

In [ ]:
# sua resposta
doc = nlp(content)
for sent in list(doc.sents)[0:10]:
  for ent in sent.ents:
        print(ent.text,' - ',ent.label_,' - ',spacy.explain(ent.label_))
  print()

## Visualização NER <a id='visualize-ner'></a>

O displaCy é uma extensão do spaCy para visualização do processo de PLN. 

Após importar a lib `displacy`, podemos usar o método `render` sobre o `doc` criado.

In [ ]:
from spacy import displacy

In [ ]:
doc = nlp(review)
displacy.render(doc, style='ent', jupyter=True)

### Exercício

Utilize o `displacy` para visualizar as entidades encontradas em algumas das sentenças do conteúdo da página do wikipedia analisada no exercício anterior.

In [ ]:
# sua resposta

# Extraindo Entidades Nomeadas

In [ ]:
import spacy
from spacy import displacy

### Modelos pré-treinados

In [ ]:
'''BAIXANDO O MODELO DA LÍNGUA PORTUGUESA'''
!python -m spacy download pt

In [ ]:
'''CLASSIFICANDO CONTEÚDO ATRAVÉS DO MODELO'''

text = 'Donald Trump, portador do cpf 064.468.404-62, chegou aos EUA no dia 01/06/2019. Donald Trump visitou Barack Obama e George Bush. Além disso, ele esteve em diversos estados do país pilotando uma ferrari.'

my_model = spacy.load('pt') #carrega o modelo
document = my_model(text) #carrega o documento

print('Original Sentence: %s\n\n' % (text))
    
displacy.render(document, jupyter=True, style='ent') #vai extrair as entidades. Pode usar outras opções, como POS, ...

### Criando seu próprio modelo

In [ ]:
import random
def train_spacy(data,iterations, base_model=None):
    TRAIN_DATA = data
    #caso tenha um modelo como base
    if base_model:
      nlp = spacy.load(base_model)
      print("Loaded model '%s'" % nlp)
    else:
      nlp = spacy.blank('pt')      
      print('Created blank model')

    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        print('ner......')
        ner = nlp.get_pipe("ner")
        
    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],         # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,       # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
    return nlp

In [ ]:
TRAIN_DATA = [('Em 15/07/1988 nasceu essa linda criança', {'entities': [(3, 13, 'Data')]}), 
              ('Rafael Mello, portador do cpf 064.468.404-62, chegou aos EUA', {'entities': [(0, 12, 'Pessoa'),(30, 44, 'CPF')]}),
              ('Data de prisão: 10/01/2018', {'entities': [(16, 26, 'Data')]}),
              ('No dia 01/02/2016 foi decretada a setença', {'entities': [(7, 17, 'Data')]}),
              ('A data da festa foi 07/05/2019', {'entities': [(20, 30, 'Data')]}),
              ('João, portador do cpf 123.456.789-65, foi encontrado', {'entities': [(0, 4, 'PER'), (18, 32, 'CPF')]}),
              ('O cpf 025.412.876-99 pertence a Maria', {'entities': [(6, 20, 'CPF')]})
             ]
#Formato - Sentença, {Entidade :[(Índice Inicial do Token, Índice Final do Token, Tipo)]} 

my_model = train_spacy(TRAIN_DATA, 20)
document = my_model(text)

print('Original Sentence: %s\n' % (text))
displacy.render(document, jupyter=True, style='ent')

Se você estiver usando um modelo existente, certifique-se de misturar exemplos de outros tipos de entidade que o spaCy reconheceu corretamente antes. Caso contrário, seu modelo pode aprender o novo tipo, mas “esquecer” o que ele sabia anteriormente. Isso também é conhecido como o problema do “esquecimento catastrófico”.

In [ ]:
my_model2 = train_spacy(TRAIN_DATA, 20, 'pt')
document = my_model2(text)

print('Original Sentence: %s\n' % (text))
displacy.render(document, jupyter=True, style='ent')

## Adicionando padrões (dicionário)

O SpaCy permite que você defina seu próprio dicionário, utilizando ou não funções linguísticas (https://spacy.io/usage/rule-based-matching)

In [ ]:
from spacy.matcher import Matcher
from spacy.tokens import Span

matcher = Matcher(my_model.vocab)
matcher.add("CARRO", None, [{"LOWER": "ferrari"}])

doc = my_model(text)
matches = matcher(doc)

for match_id, start, end in matches:
    # create a new Span for each match and use the match_id (ANIMAL) as the label
    span = Span(doc, start, end, label=match_id)
    doc.ents = list(doc.ents) + [span]  # add span to doc.ents

displacy.render(doc, jupyter=True, style='ent')

## FLAIR
Utilizaremos inicialmente o Flair Framework (https://github.com/flairNLP/flair) para nossa tarefa de NER. É uma biblioteca que implementa o estado da arte em NLP atualmente, além de permitir o uso de modelos pré-treinados de forma bastante prática e simples. Ele utiliza o Pytorch como base para definição da sua arquitetura de redes neurais. 

### Instalando

In [ ]:
!pip install flair

### Testando

In [ ]:
#import commands for flair NER
from flair.data import Sentence
from flair.models import SequenceTagger

### Modelos Pré-treinados (EN)

Você pode ver a lista completa de modelos pré-treinados aqui: https://github.com/flairNLP/flair 

In [ ]:
#Load NER Model
tagger = SequenceTagger.load('ner')

In [ ]:
#Sample text to run NER
text = 'Jackson is placed in Microsoft located in Redmond'

#passing text to sentence
sentence = Sentence(text)

# Run NER on sentence to identify Entities
tagger.predict(sentence)

# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
print(sentence.to_tagged_string())

Testando outra sentença

In [ ]:
#Sample text
text1 = 'Redmond is coming to New York city'

#passing text to sentence
sentence = Sentence(text1)

# Run NER on sentence to identify Entities
tagger.predict(sentence)

# print the entities with below command
for entity in sentence.get_spans('ner'):
    print(entity)

In [ ]:
print(sentence.to_tagged_string())

### Treinando modelo FLAIR em Portugues

O treinamento de modelos do FLAIR é bem custoso, e normalmente obtém melhores resultados utilizando embeddings pré-treinados associados a embeddings específicos para fases forward e backward do algoritmo. 

Até o momento não existem modelos prétreinados disponíveis diretamente no framework, mas algumas iniciativas já estão em andamento (https://github.com/jneto04/ner-pt)